# RNN - Many-to-one
> In this post, We will briefly cover the many-to-one type, which is one the common types of Recurrent Neural Network and its implementation in tensorflow. 

- toc: true 
- badges: true
- comments: true
- author: Chanseok Kang
- hide: true
- categories: [Python, Deep_Learning, Tensorflow-Keras]
- image:

In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

print('Tensorflow: {}'.format(tf.__version__))

plt.rcParams['figure.figsize'] = (16, 10)
plt.rc('font', size=15)

Tensorflow: 2.3.1


## Various usage of RNN
As we already discussed, RNN is used for sequence data handling. And there are several types of RNN architecture.

![various_rnn](image/various_rnn.png) {% fn 1 %}

In previous [post](https://goodboychan.github.io/chans_jupyter/python/deep_learning/tensorflow-keras/2020/10/26/02-RNN-Basic.html), we take a look **one-to-one** type, which is the basic RNN structure. And next one is **one-to-many** type. For example, if the model gets the fixed format like image as an input, it generates the sequence data. You can see the implementation on image caption application. Another type is **many-to-many** type. It gets sequence data as an inputs, and also generates the sequence data as an output. Common application of many-to-many type is machine translation.

**Many-to-one** type, which is our topic in this post, gets an sequence data as an input and generates some informatic data like labels. So we can use it for classification. Suppose that someone defines the sentiment of each sentence, and train the model with many-to-one type. And when the model gets the unseen sentence, then it will predict the intention of sentence, good or bad.

![many-to-one example](image/many-to-one.png)

The detailed explanation is like this.

WIP.

{{ 'Reference from stanford CS231n lecture note' | fndetail: 1 }}